In [1]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
st = PorterStemmer()

print('Done')

Done


In [2]:
rv = pd.read_csv('product_reviews.csv')
rv.head(3)

,Unnamed: 0,cluster,product_name,price,rating,content
0,99738,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,5,When ordering I was unsure if the holder would...
1,99739,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,1,Miss leading add it's not useful\nfor anything
2,99740,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,5,Very adaptable to all different location mounts.


# Break each review into sentences

In [3]:
# Break review into sentences
from nltk.tokenize import sent_tokenize
st = []
for sentence in rv.content:
    sent_tokenize(sentence)
    st.append(sent_tokenize(sentence))

In [4]:
rv['raw_sentence'] = st
rv.head(3)

,Unnamed: 0,cluster,product_name,price,rating,content,raw_sentence
0,99738,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,5,When ordering I was unsure if the holder would...,[When ordering I was unsure if the holder woul...
1,99739,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,1,Miss leading add it's not useful\nfor anything,[Miss leading add it's not useful\nfor anything]
2,99740,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,5,Very adaptable to all different location mounts.,[Very adaptable to all different location moun...


In [5]:
# Save to file for SA model
rv.to_csv('product_reviews(SA model).csv')

In [6]:
rv1 = rv[['cluster', 'product_name', 'price', 'raw_sentence']]
rv1.head(3)

,cluster,product_name,price,raw_sentence
0,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,[When ordering I was unsure if the holder woul...
1,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,[Miss leading add it's not useful\nfor anything]
2,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,[Very adaptable to all different location moun...


In [7]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [8]:
# Create rows for each sentence
rv2 = explode(rv1, 'raw_sentence', fill_value='')

In [9]:
# Compare with original data frame
print(rv1['raw_sentence'][0])
rv2.head(2)

['When ordering I was unsure if the holder would be strong enough to secure my JBL speaker onto the sand rail, BUT it DID!!!', 'Works GREAT and would highly recommend!', '!']


,cluster,price,product_name,raw_sentence
0,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,When ordering I was unsure if the holder would...
1,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,Works GREAT and would highly recommend!


In [10]:
# Convert sentences into lowercase
rv2['sentence'] = [x.lower() for x in rv2.raw_sentence]
rv3 = rv2.drop(columns='raw_sentence')
rv3.head(2)

,cluster,price,product_name,sentence
0,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...
1,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,works great and would highly recommend!


# POS tagging

In [11]:
# POS tagging
pos = []
for sentence in rv3.sentence:
    w = word_tokenize(sentence)
    pos.append(nltk.pos_tag(w))

In [12]:
pos[:2]

[[('when', 'WRB'),
  ('ordering', 'VBG'),
  ('i', 'NN'),
  ('was', 'VBD'),
  ('unsure', 'JJ'),
  ('if', 'IN'),
  ('the', 'DT'),
  ('holder', 'NN'),
  ('would', 'MD'),
  ('be', 'VB'),
  ('strong', 'JJ'),
  ('enough', 'RB'),
  ('to', 'TO'),
  ('secure', 'VB'),
  ('my', 'PRP$'),
  ('jbl', 'NN'),
  ('speaker', 'NN'),
  ('onto', 'IN'),
  ('the', 'DT'),
  ('sand', 'NN'),
  ('rail', 'NN'),
  (',', ','),
  ('but', 'CC'),
  ('it', 'PRP'),
  ('did', 'VBD'),
  ('!', '.'),
  ('!', '.'),
  ('!', '.')],
 [('works', 'NNS'),
  ('great', 'JJ'),
  ('and', 'CC'),
  ('would', 'MD'),
  ('highly', 'RB'),
  ('recommend', 'VB'),
  ('!', '.')]]

# Pairing

In [13]:
# Create lists for features and sentiments
ft = [[j[0] for j in i if (j[-1]=="NN" or j[-1]=="NNS" or j[-1]=="NNP" or j[-1]=="NNPS")] for i in pos]
st = [[j[0] for j in i if (j[-1]=="JJ" or j[-1]=="JJR" or j[-1]=="JJS")] for i in pos]

In [14]:
# Lemmatize the lists

ft1 = []
for k in range(len(ft)):
    words=ft[k]
    WNlemma = nltk.WordNetLemmatizer()
    refined_list = [WNlemma.lemmatize(t, pos='n') for t in words]
    ft1.append(refined_list)
    
st1 = []
for k in range(len(st)):
    words=st[k]
    WNlemma = nltk.WordNetLemmatizer()
    refined_list = [WNlemma.lemmatize(t, pos='a') for t in words]
    st1.append(refined_list)

In [15]:
print(ft[:10])
print(ft1[:10])

[['i', 'holder', 'jbl', 'speaker', 'sand', 'rail'], ['works'], [], ['miss', 'anything'], ['location', 'mounts'], ['i', 'speaker', 'strap', 'holder', 'speaker'], ['product', 'service'], ['i', 'size', 'product'], ['crisp', 'quality', 'sound'], ['size', 'bass', 'quality']]
[['i', 'holder', 'jbl', 'speaker', 'sand', 'rail'], ['work'], [], ['miss', 'anything'], ['location', 'mount'], ['i', 'speaker', 'strap', 'holder', 'speaker'], ['product', 'service'], ['i', 'size', 'product'], ['crisp', 'quality', 'sound'], ['size', 'bass', 'quality']]


In [16]:
print(st[:10])
print(st1[:10])

[['unsure', 'strong'], ['great'], [], ['useful'], ['adaptable', 'different'], ['little'], ['great', 'great'], ['sure'], ['loud', 'high'], ['good', 'loud']]
[['unsure', 'strong'], ['great'], [], ['useful'], ['adaptable', 'different'], ['little'], ['great', 'great'], ['sure'], ['loud', 'high'], ['good', 'loud']]


In [17]:
# Create feature-sentiment pairs

pairs = []

for k in range(len(st1)):
    list = [i+"/"+str(j) for i in st1[k] for j in ft1[k]]
    pairs.append(list)

print(len(pairs))
pairs[:5]

369749


[['unsure/i',
  'unsure/holder',
  'unsure/jbl',
  'unsure/speaker',
  'unsure/sand',
  'unsure/rail',
  'strong/i',
  'strong/holder',
  'strong/jbl',
  'strong/speaker',
  'strong/sand',
  'strong/rail'],
 ['great/work'],
 [],
 ['useful/miss', 'useful/anything'],
 ['adaptable/location',
  'adaptable/mount',
  'different/location',
  'different/mount']]

# Add FSP to the data frame

In [18]:
rv3['FSP'] = pairs
rv3.head(3)

,cluster,price,product_name,sentence,FSP
0,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,"[unsure/i, unsure/holder, unsure/jbl, unsure/s..."
1,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,works great and would highly recommend!,[great/work]
2,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,!,[]


In [19]:
# Drop rows without FSP
rv4 = rv3[rv3['FSP'].map(lambda d: len(d)) > 0].reset_index(drop=True)

In [20]:
rv4.head(3)

,cluster,price,product_name,sentence,FSP
0,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,"[unsure/i, unsure/holder, unsure/jbl, unsure/s..."
1,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,works great and would highly recommend!,[great/work]
2,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,miss leading add it's not useful\nfor anything,"[useful/miss, useful/anything]"


In [21]:
# The sum of FSP
rv4.FSP.str.len().sum()

2269443

In [22]:
# Create rows for each FSP
rv5 = explode(rv4, 'FSP', fill_value='')

In [23]:
print(rv5.shape[0])
rv5.head(3)

2269443


,cluster,price,product_name,sentence,FSP
0,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,unsure/i
1,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,unsure/holder
2,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,unsure/jbl


In [24]:
# Add columns for feature and sentiment
sentiment = []
feature = []

for i in range(rv5.shape[0]):
    x = rv5.FSP[i].split('/')[0]
    sentiment.append(x)
    
for i in range(rv5.shape[0]):
    x = rv5.FSP[i].split('/')[1]
    feature.append(x)


In [25]:
rv5['feature'] = feature
rv5['sentiment'] = sentiment
rv5.head(3)

,cluster,price,product_name,sentence,FSP,feature,sentiment
0,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,unsure/i,i,unsure
1,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,unsure/holder,holder,unsure
2,3,7.99,ZOEA Portable Bluetooth Speaker Mount for Golf...,when ordering i was unsure if the holder would...,unsure/jbl,jbl,unsure


In [26]:
# Save file
rv5.to_csv('FSP_without_st.csv')

# Frequency of FSP (without cleaning)

In [27]:
from collections import Counter

Counter(rv5.FSP).most_common(10)

[('great/sound', 13927),
 ('good/sound', 13281),
 ('i/i', 12321),
 ('great/speaker', 12202),
 ('good/speaker', 10146),
 ('good/i', 8121),
 ('good/quality', 7705),
 ('i/speaker', 7406),
 ('little/speaker', 7136),
 ('sound/quality', 6394)]

In [28]:
Counter(rv5.feature).most_common(10)

[('i', 163145),
 ('speaker', 154919),
 ('sound', 109745),
 ('quality', 58173),
 ('battery', 34970),
 ('volume', 33450),
 ('bass', 31639),
 ('price', 26068),
 ('music', 24760),
 ('product', 23364)]

In [29]:
Counter(rv5.sentiment).most_common(10)

[('good', 129075),
 ('great', 124408),
 ('i', 113411),
 ('little', 45263),
 ('loud', 43010),
 ('small', 38906),
 ('other', 37197),
 ('sound', 30441),
 ('easy', 30236),
 ('more', 25249)]